
# Defining Dynamic FBA model for a single species




## Import headers



In [1]:
%matplotlib inline
import cobra
import PyDSTool as dst
import pandas as pd
import matplotlib.pyplot as plt
import copy
import sys

:RESULTS:
nil:END:




## File Paths for Media spec and FBA model



In [2]:
model = cobra.io.read_sbml_model('./data/Bacteroides_sp_1_1_14.xml')
mediaFluxes = pd.read_csv('./data/VMH_HighFiber.tsv', sep='\t')

OSError: Error reading file './data/Bacteroides_sp_1_1_14.xml': failed to load external entity "./data/Bacteroides_sp_1_1_14.xml"

:RESULTS:
nil:END:




## Extract exchange reactions into variable definition and ics dictionary



In [ ]:
def cleanupname(name):
    """
    The reaction names in the model files 
    don't have brackets or parentheses. I replaced
    those found in the mediaFluxes file.
    """
    name = name.replace('[', '_LPAREN_')
    name = name.replace(']', '_RPAREN_')
    name = name.replace('(', '_LPAREN_')
    name = name.replace(')', '_RPAREN_')
    return name

"""
Initialize dictionaries for defining the ODE variables for a 
single species
"""

VarDef = dict()
ParDef = dict()
ICS_base = dict()

solution = model.optimize() # Get fluxes
all_exchanges = model.exchanges
media_derived_components = dict()

exchange_reaction_ids = [rxn.id for rxn in all_exchanges]



### Extract the reaction names and flux values into a dictionary.



In [ ]:
for i, row in mediaFluxes.iterrows():
    N = cleanupname(row.Reaction)
    media_derived_components[N] = row['Flux Value'] / (24.0*60.0) # Per minute

:RESULTS:
nil:END:




### Get the outfluxes



In [ ]:
# out_fluxes = []
# for k in all_exchanges:
#     if k.reversibility and k.id not in media_derived_components.keys():
#         out_fluxes.append(k.id)

ParDef['mu'] = solution.objective_value/60.0
VarDef['X'] =  'mu * X'
ICS_base['X'] = 1e-1

for r in all_exchanges:
    ParDef[r.id + '_Bact'] = solution.fluxes[r.id]/60
    if r.id in media_derived_components.keys():
        ParDef[r.id + '_influx'] = media_derived_components[r.id]
        VarDef[r.id] = r.id + '_influx +' + r.id + '_Bact' + ' * X'
    else: 
        VarDef[r.id] =  r.id + '_Bact' + ' * X'
    ICS_base[r.id] = 1


:RESULTS:
nil:END:




## Define PyDSTool Model



In [ ]:
TSPAN = 100

ICS = copy.deepcopy(ICS_base)
def simulateModel(ModelDS, ParDef, ICS, TSPAN, tstart):
    ModelDS.set(ics=ICS ,pars=ParDef,tdata=[tstart, tstart + TSPAN])
    P = ModelDS.compute('test').sample(dt=0.1)
    return(P)
ICS = copy.deepcopy(ICS_base)
ModelDef =dst.args(name='Bacteroides',
                  varspecs=VarDef,
                  pars=ParDef,
                  ics=ICS,
                  )

ModelDS = dst.Vode_ODEsystem(ModelDef)

P= {}
#P['X'] = [1e-1]
for k in ICS_base.keys():
    P[k] = [ICS_base[k]]

:RESULTS:
nil:END:



In [ ]:
ICS['X']


## Create Upperlimit



In [ ]:
AllPoints = []
#AllPoints.append(P)
OriginalLB = {}
model_copy = copy.deepcopy(model)
T0=0
Tfinal= 800
for r in model_copy.reactions:
    if r in model_copy.exchanges:
        OriginalLB[r.id] = r.lower_bound

NegativeCrossing = []
Lstop=len(P['X'])
i=0
MaxIter=50

while T0 < Tfinal and i < MaxIter:
    i+=1
    print(T0)
    all_exchanges = model_copy.exchanges
    for r in all_exchanges:
        model_copy.reactions.get_by_id(r.id).lower_bound = OriginalLB[r.id]*P[r.id][-1]/(0.05+P[r.id][-1])

    solution = model_copy.optimize()
#     for variable in NegativeCrossing:
#         print(variable, ' LB = ', model_copy.reactions.get_by_id(variable).lower_bound)
#         print(variable, 'flux', solution.fluxes[variable])

    ParDef['mu'] = solution.objective_value/60.0
    ICS['X'] = P['X'][-1]
    for r in all_exchanges:
        ParDef[r.id + '_Bact'] = solution.fluxes[r.id]/60 
        ICS[r.id] = P[r.id][-1]
    if T0+TSPAN > Tfinal:
        TSPAN = Tfinal-T0
    P = simulateModel(ModelDS, ParDef, ICS,TSPAN, T0)
    TSTOP = T0+TSPAN
    AllNegativeValues = {}
    Lstop=len(P['X'])-1
    NegativeCrossing = []
    
    for variable in VarDef.keys():
        if P[variable][-1] < 0.0:
            val = next((index for index,value in enumerate(P[variable]) if value < 0), None)
            if val < Lstop:
                        TSTOP = P['t'][val]
                        Lstop = val
    
    if Lstop<len(P['X'])-1:
        B={}
        for variable in VarDef.keys():
            B[variable]=P[variable][:Lstop-1]
            if P[variable][Lstop]<0:
#                 print(variable)
                B[variable][-1]=0
                NegativeCrossing.append(variable)
        B['t']=P['t'][:Lstop-1]
        P=B
    TSTOP = P['t'][-1]


                        
    
    T0 = TSTOP

    AllPoints.append(P)

In [ ]:
print(Lstop)
T=[]
X=[]
for v in VarDef.keys():
        for P in AllPoints:
                if 'h2o' not in v :
                       # print(v)
                        T = P['t']
                        X = P[v]
                        plt.plot(T,X)
#                 T = P['t']
#                 X = P['X']
#                 plt.plot(T,X)

plt.xlabel('Time (minutes)')
plt.ylabel('gdw')

In [ ]:
T=[]
X=[]
for v in VarDef.keys():
        for P in AllPoints:
#                 if 'h2o' not in v :
#                        # print(v)
#                         T = P['t']
#                         X = P[v]
#                         plt.plot(T,X)
                T = P['t']
                X = P['X']
                plt.plot(T,X)

plt.xlabel('Time (minutes)')
plt.ylabel('gdw')

In [ ]:
# for k in media_derived_components.keys():
#     if 'asn' in k:
#         print(k)

In [ ]:
# for rxn in model.exchanges:
#     print(rxn.reversibility)

This is some text